# One Hot Encoding
TJ Kim 10-15-19

Goal: Perform one-hot encoding on 'adult.csv' that holds a lot of personal and private information. The major goal is to end up with a pandas dataset with data that is easy to use for machine learning

In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

# Import Dataset
filename = "adult.csv"
df = pd.read_csv(filename, sep='\s*,\s*',engine = 'python')

# Drop unused rows
df = df.drop(['education','relationship','fnlwgt'],axis=1)
df

,age,Workclass,education-num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,14,Married-civ-spouse,Exec-managerial,White,Female,0,0,40,United-States,<=50K
6,49,Private,5,Married-spouse-absent,Other-service,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,9,Married-civ-spouse,Exec-managerial,White,Male,0,0,45,United-States,>50K
8,31,Private,14,Never-married,Prof-specialty,White,Female,14084,0,50,United-States,>50K
9,42,Private,13,Married-civ-spouse,Exec-managerial,White,Male,5178,0,40,United-States,>50K


## Perform OHE without "other" category

We process the data here to be usable with machine learning.

### Work Class Section One Hot Encoding

First we tackle workclass section and get rid of categories that are ultra-rare.

In [2]:
# Make workclass OHE
df['Workclass'] = pd.Categorical(df['Workclass'])
df_WC_Dummies = pd.get_dummies(df['Workclass'], prefix = 'Cls_Cat')

# Find the number of instances of each pandas category
df_WC_Dummies.sum(axis = 0, skipna = True) 

Cls_Cat_?                    2799
Cls_Cat_Federal-gov          1432
Cls_Cat_Local-gov            3136
Cls_Cat_Never-worked           10
Cls_Cat_Private             33906
Cls_Cat_Self-emp-inc         1695
Cls_Cat_Self-emp-not-inc     3862
Cls_Cat_State-gov            1981
Cls_Cat_Without-pay            21
dtype: int64

We will combine the "Never worked" and "without pay" category into the "?" mark category.

In [3]:
df['Workclass'] = pd.Categorical(df['Workclass'])
df_WC_Dummies = pd.get_dummies(df['Workclass'], prefix = 'Cls_Cat')

# Get rid of category without pay
df_WC_Dummies.loc[df_WC_Dummies['Cls_Cat_Without-pay'] == 1, "Cls_Cat_?"] = 1
df_WC_Dummies.loc[df_WC_Dummies['Cls_Cat_Without-pay'] == 1, "Cls_Cat_Without-pay"] = 0

# Get rid of category never worked
df_WC_Dummies.loc[df_WC_Dummies['Cls_Cat_Never-worked'] == 1, "Cls_Cat_?"] = 1
df_WC_Dummies.loc[df_WC_Dummies['Cls_Cat_Never-worked'] == 1, "Cls_Cat_Never-worked"] = 0

# Get rid of old unused pandas column
df_WC_Dummies = df_WC_Dummies.drop(columns = ['Cls_Cat_Never-worked','Cls_Cat_Without-pay'])
df_WC_Dummies.sum(axis = 0, skipna = True) 

# Place one-hot encoded columns into datafram
df = df.drop(columns = ['Workclass'])
df = pd.concat([df, df_WC_Dummies], axis=1)

### Martial Status One Hot Encoding

Next We will obtain one hot encoding for the section "Martial Status"

In [4]:
# Make one hot encoding of martial status
df['marital-status'] = pd.Categorical(df['marital-status'])
df_martial_D = pd.get_dummies(df['marital-status'], prefix = 'Martial_Cat')

# Find the number of instances of each pandas category
df_martial_D.sum(axis = 0, skipna = True) 

Martial_Cat_Divorced                  6633
Martial_Cat_Married-AF-spouse           37
Martial_Cat_Married-civ-spouse       22379
Martial_Cat_Married-spouse-absent      628
Martial_Cat_Never-married            16117
Martial_Cat_Separated                 1530
Martial_Cat_Widowed                   1518
dtype: int64

Because we do not know what married-af-spouse even means, we will collapse that with married-civ-spouse. This will help us deal with a very rare outlier.

In [5]:
# Get rid of category without pay
df_martial_D.loc[df_martial_D['Martial_Cat_Married-AF-spouse'] == 1, "Martial_Cat_Married-civ-spouse"] = 1
df_martial_D.loc[df_martial_D['Martial_Cat_Married-AF-spouse'] == 1, "Martial_Cat_Married-AF-spouse"] = 0

# Get rid of old unused pandas column
df_martial_D = df_martial_D.drop(columns = ['Martial_Cat_Married-AF-spouse'])
df_martial_D.sum(axis = 0, skipna = True) 

Martial_Cat_Divorced                  6633
Martial_Cat_Married-civ-spouse       22416
Martial_Cat_Married-spouse-absent      628
Martial_Cat_Never-married            16117
Martial_Cat_Separated                 1530
Martial_Cat_Widowed                   1518
dtype: int64

In [6]:
# Place one-hot encoded columns into datafram
df = df.drop(columns = ['marital-status'])
df = pd.concat([df, df_martial_D], axis=1)

### Native Country One Hot Encoding

In [7]:
# Make one hot encoding of country status
df['native-country'] = pd.Categorical(df['native-country'])
df_country_D = pd.get_dummies(df['native-country'], prefix = 'USA_Cat')

# Find the number of instances of each pandas category
df_country_D.sum(axis = 0, skipna = True) 

USA_Cat_?                               857
USA_Cat_Cambodia                         28
USA_Cat_Canada                          182
USA_Cat_China                           122
USA_Cat_Columbia                         85
USA_Cat_Cuba                            138
USA_Cat_Dominican-Republic              103
USA_Cat_Ecuador                          45
USA_Cat_El-Salvador                     155
USA_Cat_England                         127
USA_Cat_France                           38
USA_Cat_Germany                         206
USA_Cat_Greece                           49
USA_Cat_Guatemala                        88
USA_Cat_Haiti                            75
USA_Cat_Holand-Netherlands                1
USA_Cat_Honduras                         20
USA_Cat_Hong                             30
USA_Cat_Hungary                          19
USA_Cat_India                           151
USA_Cat_Iran                             59
USA_Cat_Ireland                          37
USA_Cat_Italy                   

Observing this data, we see that the overwhelming majority of people are from the United States. Moreover, the number of categories in this class are very large. To Prevent overfitting, we will reduce this entire category into one column; whether or not a person is based in the United States or not. Overfitting may occur because if there are too many parameters, we will need more data points to build a stable model.

In [8]:
# Loop through pandas dataframe and obtain binary whether person is US native or not
is_US = []
for index, row in df.iterrows():
    if row['native-country'] == 'United-States':
        is_US += [1]
    else:
        is_US += [0]

df['native_US'] = is_US

# Remove native country column as data is redundant now
df = df.drop(['native-country'],axis=1)

### Occupation One Hot Encoding

In [9]:
# Make one hot encoding of occupation status
df['occupation'] = pd.Categorical(df['occupation'])
df_job_D = pd.get_dummies(df['occupation'], prefix = 'Job_Cat')

# Find the number of instances of each pandas category
df_job_D.sum(axis = 0, skipna = True) 

Job_Cat_?                    2809
Job_Cat_Adm-clerical         5611
Job_Cat_Armed-Forces           15
Job_Cat_Craft-repair         6112
Job_Cat_Exec-managerial      6086
Job_Cat_Farming-fishing      1490
Job_Cat_Handlers-cleaners    2072
Job_Cat_Machine-op-inspct    3022
Job_Cat_Other-service        4923
Job_Cat_Priv-house-serv       242
Job_Cat_Prof-specialty       6172
Job_Cat_Protective-serv       983
Job_Cat_Sales                5504
Job_Cat_Tech-support         1446
Job_Cat_Transport-moving     2355
dtype: int64

Most of the jobs seem pretty evenly distributed, and there seem to be no outliers with the exception of armed forces jobs. We will simply place the armed-forces job in the category "?".

In [10]:
# Get rid of category without pay
df_job_D.loc[df_job_D['Job_Cat_Armed-Forces'] == 1, "Job_Cat_?"] = 1
df_job_D.loc[df_job_D['Job_Cat_Armed-Forces'] == 1, "Job_Cat_Armed-Forces"] = 0

# Get rid of unused column
df_job_D = df_job_D.drop(columns = ['Job_Cat_Armed-Forces'])

# Add Column to dataframe
# Place one-hot encoded columns into datafram
df = df.drop(columns = ['occupation'])
df = pd.concat([df, df_job_D], axis=1)

### Race One Hot Encoding

In [11]:
# Make one hot encoding of occupation status
df['race'] = pd.Categorical(df['race'])
df_race_D = pd.get_dummies(df['race'], prefix = 'Race_Cat')

# Find the number of instances of each pandas category
df_race_D.sum(axis = 0, skipna = True)

Race_Cat_Amer-Indian-Eskimo      470
Race_Cat_Asian-Pac-Islander     1519
Race_Cat_Black                  4685
Race_Cat_Other                   406
Race_Cat_White                 41762
dtype: int64

Since there are not that many categories for race, we will just keep all the categories. There seem to be no extreme outliers either. 

It may be interesting to remove the 'in_US' colunm for being a US native as the race distribution seems to suggest that the majority of the 'in US' people are white.

In [12]:
# Add Column to dataframe
# Place one-hot encoded columns into datafram
df = df.drop(columns = ['race'])
df = pd.concat([df, df_race_D], axis=1)

### Gender One Hot Encoding

Since gender is already binary, I will change the category to 'male' and make it binary.

In [13]:
# Loop through pandas dataframe and obtain binary whether person is US native or not
is_male = []
for index, row in df.iterrows():
    if row['sex'] == 'Male':
        is_male += [1]
    else:
        is_male += [0]

df['Is_Male'] = is_male

# Remove native country column as data is redundant now
df = df.drop(['sex'],axis=1)

### Income One Hot Encoding
Fix income into a binary value that tells us whether or not this individual makes more or less than 50k a year. If that person makes more, we will return 1, if not we will return 0.

In [14]:
# Loop through pandas dataframe and obtain binary whether person is US native or not
income_over_50k = []
for index, row in df.iterrows():
    if row['income'] == '<=50K':
        income_over_50k += [0]
    else:
        income_over_50k += [1]

df['income_over_50k'] = income_over_50k

# Remove native country column as data is redundant now
df = df.drop(['income'],axis=1)

In [15]:
df

,age,education-num,capital-gain,capital-loss,hours-per-week,Cls_Cat_?,Cls_Cat_Federal-gov,Cls_Cat_Local-gov,Cls_Cat_Private,Cls_Cat_Self-emp-inc,...,Job_Cat_Sales,Job_Cat_Tech-support,Job_Cat_Transport-moving,Race_Cat_Amer-Indian-Eskimo,Race_Cat_Asian-Pac-Islander,Race_Cat_Black,Race_Cat_Other,Race_Cat_White,Is_Male,income_over_50k
0,39,13,2174,0,40,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,50,13,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,38,9,0,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
3,53,7,0,0,40,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
4,28,13,0,0,40,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5,37,14,0,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,49,5,0,0,16,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
7,52,9,0,0,45,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
8,31,14,14084,0,50,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
9,42,13,5178,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1


## Save Pandas dataframe as CSV

Save 'df' as new csv.

In [16]:
df.to_csv('adult_encoded.csv')

In [ ]:
df.shape